# Concurrency

 * multiprocessing.Process
 * threading.Thread
 
Usage is similar: 
1. Create a derived class of Process or Thread.
2. In the constructor call constructor of Process or Thread
3. Implement `run()` method. It will be called when your Process/Thread is `start()`ed.

In [14]:
from multiprocessing import Process, Lock,RLock, Semaphore
import time

class Philosopher(Process):
    def __init__(self,i,left,right):
        self.left = left
        self.right = right
        self.id = i
        super().__init__()
    def run(self):
    
        for i in range(0,5):
            if self.id % 2 == 0:
                self.left.acquire()
                self.right.acquire()
            else:
                self.right.acquire()
                self.left.acquire()
            print("phil {} is eating".format(self.id))
            time.sleep(0.2)
            self.left.release()
            self.right.release()
            print("phil {} is thinking".format(self.id))
            time.sleep(0.1)
    

N = 5

locks = [Lock() for i in range(0,N)]

phils=[]
for i in range(0,N):
    p = Philosopher(i, locks[i], locks[i+1] if i+1 < N else locks[0])
    phils.append(p)
    
for phil in phils:
    phil.start()
    
for phil in phils:
    phil.join()


        
        

BlockingIOError: [Errno 11] Resource temporarily unavailable

## multiprocessing

`Process` is the main class. It includes synchronization related classes and data structures:

  `Lock, Semaphore, Condition, Value, Array, Queue`
  
`Process(target=`function`, args=(`arguments`)` will create a new  instance (not process yet)

calling `start()` method of the object will create the process and call the parameter function as the entry function.

In [ ]:
from multiprocessing import Process
import time

counter = 10

def hello(name):
    global counter
    for i in range(0,5):
        counter += 1
        time.sleep(0.5)
        print("hello " + name, counter)
            
p = Process(target=hello, args=("world",))

q = Process(target=hello, args=("myself",))

p.start() 
q.start()
# p, q and main process are concurrent here

p.join() # wait for p to complete
q.join() # wait for q to complete
print('counter is :',counter)
# back to single process again

 * Multiprocessing environment executes on a separate process. During process creation current set of global variables are copied in a new python interpreter and after that all work isolated.
 * `multiprocess` classes are multi-process aware. They are shared. A global lock or locks passed as parameters will be on a shared environment.

In [ ]:
''' Simple communication among two processes.
    Locks are logical entities, process do not
    have to own the lock to release it'''


# Watch this variable. There are two processes incrementing it, three reporting it
# Each process have its own copy
counter = 10
def ping(name,memut,othmut):
    global counter
    for i in range(0,5):
        # wait until my turn
        memut.acquire()
        print(name,counter)
        # tell other end it is its turn
        othmut.release()
        counter += 1

imut,omut = Lock(), Lock()

pip = Process(target=ping, args=("ping",imut,omut))
pop = Process(target=ping, args=("pong",omut,imut))

# make sure only one (ping enters first)
omut.acquire()
pip.start()
pop.start()
pip.join()
pop.join()
omut.release()
print("in main process: {}".format(counter))
    